In [128]:
import pandas as pd
import numpy as np
from pprint import pprint

In [129]:
# Importando dados
movies_df = pd.read_csv('../filmes.csv')

In [130]:
# Visualizando colunas do dataset
variables = list(movies_df.columns.values)
variables_str = '\n'.join(sorted(variables))
print('Variáveis:\n{}'.format(variables_str))
movies_df.head()
len(movies_df.columns)

Variáveis:
actor_1_facebook_likes
actor_1_name
actor_2_facebook_likes
actor_2_name
actor_3_facebook_likes
actor_3_name
aspect_ratio
budget
cast_total_facebook_likes
color
content_rating
country
director_facebook_likes
director_name
duration
facenumber_in_poster
genres
gross
imdb_score
language
movie_facebook_likes
movie_imdb_link
movie_title
num_critic_for_reviews
num_user_for_reviews
num_voted_users
plot_keywords
title_year


28

## 1. Deletando entradas com 1/3 dos valores ausentes

In [131]:
from math import ceil

num_variables = len(variables)
threshold = ceil((1/3)*num_variables)
print("Número de variáveis: {}. Limite de {} valores ausentes".format(num_variables, threshold))

    
# 1. Substituir 0's por NaN's
movies_df = movies_df.applymap(lambda x: np.nan if x == 0.0 else x)
size = len(movies_df)
# 2. Contabilizar NaNs para cada entrada
count_missing = movies_df.isnull().sum(axis='columns')
# 3. Remover
movies_df = movies_df[count_missing < threshold]
print("{} de {} entradas removidas".format(size-len(movies_df), size))

Número de variáveis: 28. Limite de 10 valores ausentes
16 de 5043 entradas removidas


## 2.0 Separando variáveis categóricas das variáveis numéricas

In [132]:
categorical_variables = ['actor_1_name', 'actor_2_name', 'actor_3_name', 'country', 'director_name', 'genres', 'language', 'movie_title', 'movie_imdb_link', 'plot_keywords', 'color', 'content_rating']
numerical_variables = [v for v in variables if v not in categorical_variables]

categorical_df = movies_df[categorical_variables]
numerical_df = movies_df[numerical_variables]

## 3. Processando variáveis categóricas

In [133]:
# Descrição do dataset categórico puro
categorical_df.describe()

actor_1_name    actor_2_name actor_3_name country     director_name  \
count             5025            5023         5016    5024              4928   
unique            2086            3026         3518      64              2387   
top     Robert De Niro  Morgan Freeman   John Heard     USA  Steven Spielberg   
freq                49              20            8    3796                26   

       genres language movie_title  \
count    5027     5018        5027   
unique    911       47        4901   
top     Drama  English  Halloween    
freq      236     4693           3   

                                          movie_imdb_link   plot_keywords  \
count                                                5027            4881   
unique                                               4903            4751   
top     http://www.imdb.com/title/tt2638144/?ref_=fn_t...  based on novel   
freq                                                    3               4   

        color content_rating  
count    5011           4739  
unique      2             18  
top     Color              R  
freq     4802           2118

In [134]:
# Amostra do dataset
categorical_df.head()

actor_1_name      actor_2_name          actor_3_name country  \
0      CCH Pounder  Joel David Moore             Wes Studi     USA   
1      Johnny Depp     Orlando Bloom        Jack Davenport     USA   
2  Christoph Waltz      Rory Kinnear      Stephanie Sigman      UK   
3        Tom Hardy    Christian Bale  Joseph Gordon-Levitt     USA   
5     Daryl Sabara   Samantha Morton          Polly Walker     USA   

       director_name                           genres language  \
0      James Cameron  Action|Adventure|Fantasy|Sci-Fi  English   
1     Gore Verbinski         Action|Adventure|Fantasy  English   
2         Sam Mendes        Action|Adventure|Thriller  English   
3  Christopher Nolan                  Action|Thriller  English   
5     Andrew Stanton          Action|Adventure|Sci-Fi  English   

                                 movie_title  \
0                                    Avatar    
1  Pirates of the Caribbean: At World's End    
2                                   Spectre    
3                     The Dark Knight Rises    
5                               John Carter    

                                     movie_imdb_link  \
0  http://www.imdb.com/title/tt0499549/?ref_=fn_t...   
1  http://www.imdb.com/title/tt0449088/?ref_=fn_t...   
2  http://www.imdb.com/title/tt2379713/?ref_=fn_t...   
3  http://www.imdb.com/title/tt1345836/?ref_=fn_t...   
5  http://www.imdb.com/title/tt0401729/?ref_=fn_t...   

                                       plot_keywords  color content_rating  
0             avatar|future|marine|native|paraplegic  Color          PG-13  
1  goddess|marriage ceremony|marriage proposal|pi...  Color          PG-13  
2                bomb|espionage|sequel|spy|terrorist  Color          PG-13  
3  deception|imprisonment|lawlessness|police offi...  Color          PG-13  
5  alien|american civil war|male nipple|mars|prin...  Color          PG-13

### 3.0 Uniformizando o texto

In [135]:
# Tirando todos os espaços em branco antes e depois das palavras
categorical_df = categorical_df.applymap(lambda x: x.strip() if type(x) is str else x)

### 3.1 Tratando valores ausentes
Estratégias:
1. Nomes dos atores: Deixar ausente
2. País: 
3. Diretor
4. Idioma:
5. Título: deixar ausente
6. Keywords: deixar ausente
8. Link do imdb: deixar ausente
9. Cor: colorido se número de likes no fb != nan

#### Atores

In [136]:
missing = categorical_df[categorical_df['actor_1_name'].isna() & categorical_df['actor_2_name'].isna() | categorical_df['actor_3_name'].isna()]
missing.describe()

actor_1_name   actor_2_name  actor_3_name country   director_name  \
count                9              7           0.0      11              11   
unique               9              7           0.0       7              11   
top     Jacques Perrin  Bobby Kendall           NaN     USA  Jacques Perrin   
freq                 1              1           NaN       5               1   

             genres language  movie_title  \
count            11       11           11   
unique            7        2           11   
top     Documentary  English  All Is Lost   
freq              5       10            1   

                                          movie_imdb_link  \
count                                                  11   
unique                                                 11   
top     http://www.imdb.com/title/tt0302674/?ref_=fn_t...   
freq                                                    1   

                                      plot_keywords  color content_rating  
count                                             9     11              8  
unique                                            9      1              5  
top     boat|container|sea|shipping container|storm  Color      Not Rated  
freq                                              1     11              3

#### País

In [137]:
country_df = categorical_df.country;
country_df.describe()

count     5024
unique      64
top        USA
freq      3796
Name: country, dtype: object

In [138]:
print("Tabela de Frequências")
country_df.value_counts().head()

Tabela de Frequências


USA        3796
UK          447
France      154
Canada      126
Germany      97
Name: country, dtype: int64

In [139]:
## Valores Ausentes
categorical_df[categorical_df['country'].isna()]

actor_1_name          actor_2_name  \
2370  Nicole 'Snooki' Polizzi  Jenni 'Jwoww' Farley   
3397           Dominic Cooper         Joseph Gilgun   
4021             Chris Brochu            Jeff Fahey   

                   actor_3_name country      director_name  \
2370  Paul 'Pauly D' DelVecchio     NaN                NaN   
3397                 Ruth Negga     NaN                NaN   
4021                Rita Wilson     NaN  Daniel Petrie Jr.   

                               genres language       movie_title  \
2370  Comedy|Drama|Reality-TV|Romance  English  Gone, Baby, Gone   
3397  Adventure|Drama|Fantasy|Mystery  English          Preacher   
4021                   Drama|Thriller  English       Dawn Patrol   

                                        movie_imdb_link  \
2370  http://www.imdb.com/title/tt1697237/?ref_=fn_t...   
3397  http://www.imdb.com/title/tt5016504/?ref_=fn_t...   
4021  http://www.imdb.com/title/tt2073661/?ref_=fn_t...   

                                          plot_keywords  color content_rating  
2370                                          lifestyle  Color          TV-14  
3397  heaven and hell|preacher|supernatural|vampire|...  Color          TV-MA  
4021  desert|held at gunpoint|marine|revenge|sex on ...  Color            NaN

In [140]:
# Ajustando todos valores ausentes manualmente, baseando-se no país
categorical_df.loc[2370, 'country'] = "USA"
categorical_df.loc[2370, 'director_name'] = "Ben Affleck"

categorical_df.loc[3397, 'country'] = "USA"
categorical_df.loc[3397, 'director_name'] = "Sam Catlin"

categorical_df.loc[4021, 'country'] = "USA"

#### Diretor

In [141]:
categorical_df.loc[categorical_df['director_name'].isna(), 'movie_title']

177                     Miami Vice
260                     The A-Team
404                       Hannibal
459                      Daredevil
479                      Bewitched
537                    Constantine
543                           Life
593                  Sleepy Hollow
645              Last Man Standing
685                    The Missing
757            Rules of Engagement
816     Sabrina, the Teenage Witch
826               Sex and the City
833               Anger Management
857                  Stargate SG-1
962                    Unforgotten
994               A Touch of Frost
1007                       Twisted
1023                      Defiance
1035                     Outlander
1133                  The Returned
1175                 McHale's Navy
1265                        Arthur
1346         3rd Rock from the Sun
1404                     Rush Hour
1444                 Hit the Floor
1499                        Luther
1620           Friday Night Lights
1659                

#### Gêneros e keywords (ambos OK)

In [142]:
genres = categorical_df['genres'].astype(str)
genres[genres=='nan']

Series([], Name: genres, dtype: object)

#### Idioma

In [143]:
categorical_df['language'].value_counts().head()

English     4693
French        73
Spanish       40
Hindi         28
Mandarin      26
Name: language, dtype: int64

In [144]:
categorical_df.loc[categorical_df['language'].isna()]

actor_1_name    actor_2_name       actor_3_name country  \
3086                Jon Gries  Taylor Handley         Trent Ford     USA   
3539          Debi Derryberry    Kate Higgins     Cindy Robinson     USA   
3869               Sid Caesar     Dom DeLuise  Bernadette Peters     USA   
4110  William Morgan Sheppard      Kevin Gage      Brianna Brown     USA   
4409             Matthew Ziff      T.J. Storm         Sam Medina     USA   
4630           Justin Baldoni      Luke Perry      Leonor Varela     USA   
4810             Lillian Gish       Mae Marsh        Walter Long     USA   
4885             John Gilbert    Renée Adorée       Claire Adams     USA   
4958             Stephen Carr  Johnnie Walker          Mary Carr     USA   

           director_name                                             genres  \
3086    Christopher Cain                      Drama|History|Romance|Western   
3539        Richard Rich  Action|Adventure|Animation|Comedy|Drama|Family...   
3869          Mel Brooks                                     Comedy|Romance   
4110  Michael Landon Jr.                               Drama|Family|Western   
4409      John Stockwell                                             Action   
4630     Jonathan Meyers                                              Drama   
4810       D.W. Griffith                                  Drama|History|War   
4885          King Vidor                                  Drama|Romance|War   
4958   Harry F. Millarde                                        Crime|Drama   

     language                                        movie_title  \
3086      NaN                                     September Dawn   
3539      NaN  Alpha and Omega 4: The Legend of the Saw Tooth...   
3869      NaN                                       Silent Movie   
4110      NaN                                 Love's Abiding Joy   
4409      NaN                               Kickboxer: Vengeance   
4630      NaN                                        A Fine Step   
4810      NaN   Intolerance: Love's Struggle Throughout the Ages   
4885      NaN                                     The Big Parade   
4958      NaN                     Over the Hill to the Poorhouse   

                                        movie_imdb_link  \
3086  http://www.imdb.com/title/tt0473700/?ref_=fn_t...   
3539  http://www.imdb.com/title/tt4061848/?ref_=fn_t...   
3869  http://www.imdb.com/title/tt0075222/?ref_=fn_t...   
4110  http://www.imdb.com/title/tt0785025/?ref_=fn_t...   
4409  http://www.imdb.com/title/tt3082898/?ref_=fn_t...   
4630  http://www.imdb.com/title/tt1604100/?ref_=fn_t...   
4810  http://www.imdb.com/title/tt0006864/?ref_=fn_t...   
4885  http://www.imdb.com/title/tt0015624/?ref_=fn_t...   
4958  http://www.imdb.com/title/tt0011549/?ref_=fn_t...   

                                          plot_keywords            color  \
3086           massacre|mormon|settler|utah|wagon train            Color   
3539                blindness|cave|spirit|wolf|wolf cub              NaN   
3869  black comedy|friend|modern silent movie|silent...            Color   
4110             19th century|faith|mayor|ranch|sheriff            Color   
4409                                                NaN              NaN   
4630                                                NaN              NaN   
4810    huguenot|intolerance|medicis|protestant|wedding  Black and White   
4885  chewing gum|climbing a tree|france|translation...  Black and White   
4958    family relationships|gang|idler|poorhouse|thief  Black and White   

     content_rating  
3086              R  
3539            NaN  
3869             PG  
4110             PG  
4409            NaN  
4630             PG  
4810      Not Rated  
4885      Not Rated  
4958            NaN

In [145]:
#Todos os que faltam são americanos, logo são em inglês
categorical_df.loc[categorical_df['language'].isna(), 'language'] = 'English'

#### Classificação Indicativa

In [146]:
categorical_df.loc[categorical_df['content_rating'].isna()]

actor_1_name              actor_2_name  \
98             Mark Chinnery         Shin'ya Tsukamoto   
199             Rupert Grint               Dave Legeno   
204            Mark Chinnery         Shin'ya Tsukamoto   
206             Rupert Grint                Toby Jones   
242              Alain Delon           Santiago Segura   
489            Nissim Renard              Roxane Duran   
543               Adam Arkin              Brent Sexton   
685            Jason Flemyng             James Nesbitt   
810                  Li Gong                Aaron Kwok   
833             Barry Corbin            Noureen DeWulf   
962             Bernard Hill               Gemma Jones   
994              David Jason           Bruce Alexander   
1007              Grey Damon           Brittany Curran   
1044              P.J. Soles             Lochlyn Munro   
1178          Jamel Debbouze             Youssef Hajdi   
1329        Tamannaah Bhatia            Anushka Shetty   
1347         Blythe Auffarth            Bianca Collins   
1382           Yuliya Snigir          Fedor Bondarchuk   
1444        Jodi Lyn O'Keefe            Kimberly Elise   
1510              Bill Cobbs               Aisha Tyler   
1527             Kyle Hebert            Brianne Brozey   
1710   Ólafur Darri Ólafsson  Ingvar Eggert Sigurðsson   
1724         Louise Bourgoin        Sandrine Kiberlain   
1730                Paz Vega            Frank Langella   
1731   Konstantin Khabenskiy          Fedor Bondarchuk   
1793         Viggo Mortensen               Elena Anaya   
1818          Jackie Gleason                Art Carney   
1831     Kim-Marie Woodhouse                Amy Carson   
1878           Jason Flemyng             Igor Jijikine   
1977               Dany Boon             Malonn Lévana   
...                      ...                       ...   
4939                Lisa Ann                Jesse Jane   
4940             Ariel Hsing              Xinhua Jiang   
4949       Christopher Judge             David Hewlett   
4950       Christopher Judge             David Hewlett   
4952          Philippe Leroy               Macha Méril   
4954            Richard Moll             Frank Gerrish   
4958            Stephen Carr            Johnnie Walker   
4968    Claire Gordon-Harper                  Hugh Mun   
4972            Noam Chomsky             Seth Ackerman   
4976  Michelle Simone Miller           Steve Diasparra   
4989          Corbin Bernsen              Chelse Swain   
4991                Dan Novy            David Mersault   
4994          Taliesin Jaffe            Greg Aronowitz   
4996          Katrina Bowden                 Jon Gries   
4999            Barry Corbin             Johnny Walter   
5002         Julianne Gabert             Thomas Brophy   
5010         Andrew Bujalski          Kate Dollenmayer   
5013                Franky G            Panchito Gómez   
5016             Tjasa Ferme              Mikaal Bates   
5017          Kristen Seavey             Suzi Lorraine   
5018          Shannen Fields               Lisa Arnold   
5020           Robbie Barnes           Alana Kaniewski   
5022              Paul Ogola             Olwenya Maina   
5023        Zoe Lister-Jones             Heather Burns   
5028       Patrick O'Donnell             Michael Parle   
5029            Kôji Yakusho             Anna Nakagawa   
5030     Tatiana Suarez-Pico            Michael Cortez   
5031           Julianna Pitt                Joe Coffey   
5032            Peter Greene         Stanley B. Herman   
5038             Eric Mabius             Daphne Zuniga   

                  actor_3_name      country          director_name  \
98                Atsuko Maeda        Japan           Hideaki Anno   
199               Ralph Ineson           UK             Matt Birch   
204               Atsuko Maeda        Japan           Hideaki Anno   
206               Alfred Enoch           UK             Matt Birch   
242            Vanessa Hessler       France     Frédéric Forestier   

In [147]:
rating_freq = categorical_df['content_rating'].value_counts()
rating_labels = rating_freq.index
rating_freqs = rating_freq.values
normalization_factor = sum(rating_freqs) #normalizando
weigths = [i/normalization_factor for i in rating_freqs]
rating_freq

R            2118
PG-13        1460
PG            701
Not Rated     116
G             112
Unrated        62
Approved       55
TV-14          30
TV-MA          20
X              13
TV-PG          13
TV-G           10
Passed          9
NC-17           7
GP              6
M               5
TV-Y            1
TV-Y7           1
Name: content_rating, dtype: int64

In [148]:
# Estratégia: Atribuir a classificação mais frequente de cada diretor
from numpy.random import choice
missing = categorical_df.loc[categorical_df['content_rating'].isna()]
for index, row in missing.iterrows():
    ratings = categorical_df.loc[categorical_df['director_name'] == row['director_name'], 'content_rating'].value_counts()
    if len(ratings)> 0:
        rating = ratings.index[0]
    else:
        rating = choice(rating_labels, p=weigths) # se não achar realiza sorteio poderado com frequência
    categorical_df.loc[index, 'content_rating'] = rating

#### Cor

In [60]:
# Visualização dos dados antes de tratar dados ausentes
color_df = categorical_df['color']
print("Frequências:\n{}".format(color_df.astype(str).value_counts()))
animations = movies_df['genres'].apply(lambda x: "Animation" in x if type(x) is str else x)

Frequências:
Color               4802
 Black and White     209
nan                   16
Name: color, dtype: int64


0       False
1       False
2       False
3       False
5       False
6       False
7        True
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
30      False
        ...  
5012    False
5013    False
5014    False
5015    False
5016    False
5017    False
5018    False
5019    False
5020    False
5021    False
5022    False
5023    False
5024    False
5025    False
5026    False
5027    False
5028    False
5029    False
5030    False
5031    False
5032    False
5033    False
5034    False
5035    False
5036    False
5037    False
5038    False
5039    False
5041    False
5042    False
Name: genres, Length: 5027, dtype: bool

In [26]:
movies_df.loc[movies_df['color'] == 'Color', 'title_year'].value_counts()
movies_df.loc[movies_df['title_year'] == 1937, 'movie_title']
movies_df.loc[4449] # Animações dps de 1937 são coloridas
movies_df[movies_df['']]

color                                                                    Color
director_name                                                 William Cottrell
num_critic_for_reviews                                                     145
duration                                                                    83
director_facebook_likes                                                    NaN
actor_3_facebook_likes                                                      31
actor_2_name                                                     Billy Gilbert
actor_1_facebook_likes                                                      82
gross                                                              1.84925e+08
genres                                        Animation|Family|Fantasy|Musical
actor_1_name                                                 Adriana Caselotti
movie_title                                   Snow White and the Seven Dwarfs 
num_voted_users                                     

In [149]:
#Tratando dados ausentes
is_color = numerical_df['movie_facebook_likes'] > 0

# Não tem valor e é colorido
color_df = categorical_df[is_color & categorical_df['color'].isna()]
color_df['color'] = 'Color'
categorical_df.update(color_df)

# Não tem valor e é preto e branco
color_df = categorical_df[~is_color & categorical_df['color'].isna()]
color_df['color'] = 'Black and White'

# Atualizando as entradas do dataframe
categorical_df.update(color_df)
categorical_df['color'].describe()

print("Valores presentes: {}".format(set(categorical_df['color'].values)))

Valores presentes: {'Color', 'Black and White'}


/home/dcandrade/.anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/dcandrade/.anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


### 3.2 Codificando colunas em formato de lista (gênero e keywords)

#### Gêneros

In [119]:
#Processando gênero
# Transformando string em lista
genres =  categorical_df.genres.apply(lambda x: x.replace('|', ' ').replace('-', ''))
genres.head()

0    Action Adventure Fantasy SciFi
1          Action Adventure Fantasy
2         Action Adventure Thriller
3                   Action Thriller
5            Action Adventure SciFi
Name: genres, dtype: object

In [120]:
vocabulary = set()
genres.apply(lambda x: vocabulary.update(x.split(' ')))

from sklearn.feature_extraction.text import CountVectorizer
genre_vectorizer = CountVectorizer()
genre_vectorizer.fit(vocabulary)
vocabulary = sorted(genre_vectorizer.vocabulary_)
vocabulary

['action',
 'adventure',
 'animation',
 'biography',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'family',
 'fantasy',
 'filmnoir',
 'history',
 'horror',
 'music',
 'musical',
 'mystery',
 'news',
 'realitytv',
 'romance',
 'scifi',
 'short',
 'sport',
 'thriller',
 'war',
 'western']

In [126]:
vect_genres = genre_vectorizer.transform(genres.values)
genres_binary_df = pd.DataFrame(vect_genres.toarray())
genres_binary_df.rename(lambda x: 'genre_'+vocabulary[int(x)], axis='columns', inplace=True)
genres_binary_df.head()

genre_action  genre_adventure  genre_animation  genre_biography  \
0             1                1                0                0   
1             1                1                0                0   
2             1                1                0                0   
3             1                0                0                0   
4             1                1                0                0   

   genre_comedy  genre_crime  genre_documentary  genre_drama  genre_family  \
0             0            0                  0            0             0   
1             0            0                  0            0             0   
2             0            0                  0            0             0   
3             0            0                  0            0             0   
4             0            0                  0            0             0   

   genre_fantasy      ...        genre_mystery  genre_news  genre_realitytv  \
0              1      ...                    0           0                0   
1              1      ...                    0           0                0   
2              0      ...                    0           0                0   
3              0      ...                    0           0                0   
4              0      ...                    0           0                0   

   genre_romance  genre_scifi  genre_short  genre_sport  genre_thriller  \
0              0            1            0            0               0   
1              0            0            0            0               0   
2              0            0            0            0               1   
3              0            0            0            0               1   
4              0            1            0            0               0   

   genre_war  genre_western  
0          0              0  
1          0              0  
2          0              0  
3          0              0  
4          0              0  

[5 rows x 25 columns]

In [150]:
# Colocando de volta no Dataframe
categorical_df.drop('genres', axis='columns', inplace=True)
categorical_df = categorical_df.join(genres_binary_df)
categorical_df.head()

actor_1_name      actor_2_name          actor_3_name country  \
0      CCH Pounder  Joel David Moore             Wes Studi     USA   
1      Johnny Depp     Orlando Bloom        Jack Davenport     USA   
2  Christoph Waltz      Rory Kinnear      Stephanie Sigman      UK   
3        Tom Hardy    Christian Bale  Joseph Gordon-Levitt     USA   
5     Daryl Sabara   Samantha Morton          Polly Walker     USA   

       director_name language                               movie_title  \
0      James Cameron  English                                    Avatar   
1     Gore Verbinski  English  Pirates of the Caribbean: At World's End   
2         Sam Mendes  English                                   Spectre   
3  Christopher Nolan  English                     The Dark Knight Rises   
5     Andrew Stanton  English                               John Carter   

                                     movie_imdb_link  \
0  http://www.imdb.com/title/tt0499549/?ref_=fn_t...   
1  http://www.imdb.com/title/tt0449088/?ref_=fn_t...   
2  http://www.imdb.com/title/tt2379713/?ref_=fn_t...   
3  http://www.imdb.com/title/tt1345836/?ref_=fn_t...   
5  http://www.imdb.com/title/tt0401729/?ref_=fn_t...   

                                       plot_keywords  color      ...        \
0             avatar|future|marine|native|paraplegic  Color      ...         
1  goddess|marriage ceremony|marriage proposal|pi...  Color      ...         
2                bomb|espionage|sequel|spy|terrorist  Color      ...         
3  deception|imprisonment|lawlessness|police offi...  Color      ...         
5  alien|american civil war|male nipple|mars|prin...  Color      ...         

  genre_mystery  genre_news  genre_realitytv  genre_romance  genre_scifi  \
0           0.0         0.0              0.0            0.0          1.0   
1           0.0         0.0              0.0            0.0          0.0   
2           0.0         0.0              0.0            0.0          0.0   
3           0.0         0.0              0.0            0.0          0.0   
5           0.0         0.0              0.0            1.0          0.0   

   genre_short  genre_sport  genre_thriller  genre_war  genre_western  
0          0.0          0.0             0.0        0.0            0.0  
1          0.0          0.0             0.0        0.0            0.0  
2          0.0          0.0             1.0        0.0            0.0  
3          0.0          0.0             1.0        0.0            0.0  
5          0.0          0.0             0.0        0.0            0.0  

[5 rows x 36 columns]

#### Keywords

In [18]:
# Processando keywords
keywords = categorical_df.plot_keywords.apply(lambda x: x.split('|') if type(x) is str else 'None')
keywords.head()

0         [avatar, future, marine, native, paraplegic]
1    [goddess, marriage ceremony, marriage proposal...
2            [bomb, espionage, sequel, spy, terrorist]
3    [deception, imprisonment, lawlessness, police ...
5    [alien, american civil war, male nipple, mars,...
Name: plot_keywords, dtype: object

In [19]:
# Quebrando o array em colunas
keywords = keywords.apply(pd.Series)
keywords.rename(lambda x: 'keyword_{}'.format(x), axis='columns', inplace=True)
keywords = keywords.astype(str) # nan is the value for missing

In [20]:
#Convertendo as categorias para inteiros
all_keywords = set()
keywords.applymap(lambda x: all_keywords.add(x))
keywords_encoder = LabelEncoder()
keywords_encoder.fit(list(all_keywords))
keywords = keywords.apply(keywords_encoder.transform)
keywords.head()

keyword_0  keyword_1  keyword_2  keyword_3  keyword_4
0        515       2866       4301       4759       5158
1       3026       4314       4317       5313       6522
2        875       2327       6334       6785       7202
3       1894       3565       3964       5405       7206
5        216        264       4249       4319       5544

In [21]:
# Colocando de volta no Dataframe
categorical_df.drop('plot_keywords', axis='columns', inplace=True)
categorical_df = categorical_df.join(keywords)
categorical_df.head()

actor_1_name      actor_2_name          actor_3_name country  \
0      CCH Pounder  Joel David Moore             Wes Studi     USA   
1      Johnny Depp     Orlando Bloom        Jack Davenport     USA   
2  Christoph Waltz      Rory Kinnear      Stephanie Sigman      UK   
3        Tom Hardy    Christian Bale  Joseph Gordon-Levitt     USA   
5     Daryl Sabara   Samantha Morton          Polly Walker     USA   

       director_name language                               movie_title  \
0      James Cameron  English                                    Avatar   
1     Gore Verbinski  English  Pirates of the Caribbean: At World's End   
2         Sam Mendes  English                                   Spectre   
3  Christopher Nolan  English                     The Dark Knight Rises   
5     Andrew Stanton  English                               John Carter   

                                     movie_imdb_link  color  genre_0  \
0  http://www.imdb.com/title/tt0499549/?ref_=fn_t...  Color        0   
1  http://www.imdb.com/title/tt0449088/?ref_=fn_t...  Color        0   
2  http://www.imdb.com/title/tt2379713/?ref_=fn_t...  Color        0   
3  http://www.imdb.com/title/tt1345836/?ref_=fn_t...  Color        0   
5  http://www.imdb.com/title/tt0401729/?ref_=fn_t...  Color        0   

     ...      genre_3  genre_4  genre_5  genre_6  genre_7  keyword_0  \
0    ...           19       25       25       25       25        515   
1    ...           25       25       25       25       25       3026   
2    ...           25       25       25       25       25        875   
3    ...           25       25       25       25       25       1894   
5    ...           25       25       25       25       25        216   

   keyword_1  keyword_2  keyword_3  keyword_4  
0       2866       4301       4759       5158  
1       4314       4317       5313       6522  
2       2327       6334       6785       7202  
3       3565       3964       5405       7206  
5        264       4249       4319       5544  

[5 rows x 22 columns]